Compute the Monte Carlo price of European options with the Sun Yu model

In [27]:
import cupy as cp
import numpy as np

## 1.1 Set default device as CUDA

## 1.2 Parameters of the model

In [9]:
# Set parameter values for multifactor stochastic volatility model
# Recchioni, Sun, An explicitly solvable Heston model with stochastic interest rate,
# European Journal of Operational Research 249 (1), 359-377, 2016.

# Market parameters
S0 = 1; # spot exchange rate
r_0 = cp.array([0.02,0.01]); # spot interest rates r_{i0},r_{j0}

# Contract parameters
T = 1; # maturity
K = 1; # strike price

# Model parameters

param_alpha = 0.5;
d = 2; # number of volatility factors

# Volatility coefficients or weights
a_i = cp.array([0.6650, 1.0985]);
a_j = cp.array([1.6177, 1.3588]);

# Mean-reversion rate (or strength) of the volatility
chi = cp.array([0.9418,1.7909]);

# Initial volatility
v_0 = cp.array([0.1244,0.0391]);

# Long-term average of the volatility
v_bar = cp.array([0.037,0.0909]);

# Volatility of volatility
gamma = cp.array([0.4912,1]);

# Interest rate coefficients or weights
b_i = cp.array([1.0000004,0.000000]);
b_j = cp.array([0.000000,0.0000006]);

# Mean-reversion rate (or strength) of the interest rate
Lambda = cp.array([0.01,0.02]); # lambda_i,lambda_j

# Long-term average of the interest rate
r_bar = cp.array([0.02,0.01]); # \bar{r}_i,\bar{r}_j

# Volatility of the interest rate
eta = cp.array([0.001,0.002]); # \eta_i,\eta_j

# Correlations
rho_v = cp.array([-0.5231,-0.398]);
rho_r = cp.array([-0.23,-0.81]);

## 2.1 Monte Carlo simulation

In [10]:
# Algorithm parameters
nsteps = 10;
nblocks = 200;
npaths = 1000;

In [11]:
from cupy.array_api import float32

# Monte Carlo
dt = T/nsteps;
VcMC = cp.zeros((nblocks), dtype=float32);
VpMC = cp.zeros((nblocks), dtype=float32);
for block in range (nblocks):
    MC_result = cp.zeros((nsteps+1,npaths), dtype=float32);
    VcMCb = cp.zeros((1,npaths), dtype=float32);
    VpMCb = cp.zeros((1,npaths), dtype=float32);
    for path in range(npaths):
        # Increments of the arithmetic Brownian motion X(t) = log(S(t)/S(0))
        # dX = muABM*dt + sigma*sqrt(dt)*randn(ndates,nsample);
        # dS/S = (r_0 - r_i)*dt - a_i * diag_v^0.5 * dW_v - b_i * diag_r^alpha * dW_r

        # Generate a random path using the above model
        # Volatility part
        # Model variables
        v = cp.zeros((nsteps+1,d), dtype=float32);
        v[0,] = v_0;
        # corr (dW_v, dZ_v) = rho_v
        dW_v_1 = cp.random.randn(nsteps,d, dtype=float32);
        dW_v_2 = cp.random.randn(nsteps,d, dtype=float32);
        dW_v_3 = cp.matmul(dW_v_1,cp.diag(rho_v)) + cp.matmul(dW_v_2,cp.diag((1-rho_v**2)**0.5));

        dW_v = dW_v_1 * cp.sqrt(dt);
        dZ_v = dW_v_3 * cp.sqrt(dt);

        v_ref = cp.zeros((1,d), dtype=float32);
        # dv = chi * (v_bar - v) * dt + gamma * \sqrt(v) * dZ_v
        for steps in range(nsteps):
            v[steps+1,] = cp.maximum(v[steps,] + cp.matmul((v_bar - v[steps,]) , cp.diag(chi)) * dt
                              + cp.matmul(cp.sqrt(v[steps,]) * dZ_v[steps,], cp.diag(gamma)),v_ref);

        # Interest rate part
        # Model variables
        r = cp.zeros((nsteps+1,2), dtype=float32);
        r[0,] = r_0;

        # corr (dW_r_i, dZ_r_i) = rho_r_i
        dW_r_1 = cp.random.randn(nsteps,2, dtype=float32);
        dW_r_2 = cp.random.randn(nsteps,2, dtype=float32);
        dW_r_3 = cp.matmul(dW_r_1,cp.diag(rho_r)) + cp.matmul(dW_r_2,cp.diag((1-rho_r**2)**0.5));

        dW_r = dW_r_1 * cp.sqrt(dt);
        dZ_r = dW_r_3 * cp.sqrt(dt);

        r_ref = cp.zeros((1,2));
        # dr = lambda * (r_bar - r) * dt + eta * r^alpha * dZ_r
        for steps in range(nsteps):
            r[steps+1,] = cp.maximum(r[steps,] + cp.matmul((r_bar - r[steps,]) , cp.diag(Lambda)) * dt
                              + cp.matmul(cp.sqrt(r[steps,]) * dZ_r[steps,], cp.diag(eta)),r_ref);

        sum_v_1 = cp.zeros(nsteps, dtype=float32);
        sum_v_2 = cp.zeros(nsteps, dtype=float32);
        sum_r_1 = cp.zeros(nsteps, dtype=float32);
        sum_r_2 = cp.zeros(nsteps, dtype=float32);
        mu = cp.zeros(nsteps, dtype=float32);
        x = cp.zeros((nsteps+1,1), dtype=float32);

        # Valuation for dx
        for steps in range(nsteps):
            # Block for MuYu
            # sum_v_1(steps) = (a_i(1)^2 - a_j(1)^2) * v_1(steps) + (a_i(2)^2 - a_j(2)^2) * v_2(steps);
            sum_v_1[steps] = cp.matmul(v[steps,] , (a_i**2 - a_j**2));
            # sum_r_1(steps) = b_i^2 * r_i(steps)^(2*param_alpha) - b_j^2 * r_j(steps)^(2*param_alpha);
            sum_r_1[steps] = cp.matmul(r[steps,] , (b_i**2 - b_j**2));

            mu[steps] = r[steps,0] - r[steps,1] + 0.5 * (sum_v_1[steps] + sum_r_1[steps]);

            # Block for v
            sum_v_2[steps] = cp.matmul(v[steps,:]**0.5 * dW_v[steps,] , (a_i-a_j));###

            # Block for r
            sum_r_2[steps] = cp.matmul(r[steps,]**param_alpha * dW_r[steps,],(b_i-b_j));###

            # dx = (r_0 - r_i)*dt - a_i * diag_v^0.5 * dW_v - b_i * diag_r^alpha * dW_r
            x[steps+1] = x[steps] + mu[steps]*dt + sum_v_2[steps] + sum_r_2[steps];

        # Calculate the Smax and Smin

        S_end = S0*cp.exp(x[-1]);

        payoffs_call = max(S_end - K,0);
        payoffs_put = max(K - S_end,0);

        VcMCb[0,path] = cp.exp(-r_0[1]*T)*payoffs_call;
        VpMCb[0,path] = cp.exp(-r_0[1]*T)*payoffs_put;

       # MC_result(1:nsteps+1,path) = S0*exp(x_i_j);

    VcMC[block] = cp.mean(VcMCb);
    VpMC[block] = cp.mean(VpMCb);
    print(block);


VcMC_result = cp.mean(VcMC);
VpMC_result = cp.mean(VpMC);
scMC = cp.sqrt(cp.var(VcMC)/nblocks);
spMC = cp.sqrt(cp.var(VpMC)/nblocks);

0
1
2
3


KeyboardInterrupt: 

In [ ]:
VcMC_result


In [ ]:
VpMC_result

In [ ]:
cp.zeros(10)

In [28]:
def simulation_path(d,dt,v_0,r_0,rho_r,rho_v,nsteps,S0,K,eta,gamma):
    v = cp.zeros((nsteps+1,d), dtype=float32);
    v[0,] = v_0;
    # corr (dW_v, dZ_v) = rho_v
    dW_v_1 = cp.random.randn(nsteps,d, dtype=float32);
    dW_v_2 = cp.random.randn(nsteps,d, dtype=float32);
    dW_v_3 = cp.matmul(dW_v_1,cp.diag(rho_v)) + cp.matmul(dW_v_2,cp.diag((1-rho_v**2)**0.5));

    dW_v = dW_v_1 * cp.sqrt(dt);
    dZ_v = dW_v_3 * cp.sqrt(dt);

    v_ref = cp.zeros((1,d), dtype=float32);
    # dv = chi * (v_bar - v) * dt + gamma * \sqrt(v) * dZ_v
    for steps in range(nsteps):
        v[steps+1,] = cp.maximum(v[steps,] + cp.matmul((v_bar - v[steps,]) , cp.diag(chi)) * dt  + cp.matmul(cp.sqrt(v[steps,]) * dZ_v[steps,], cp.diag(gamma)),v_ref);

        # Interest rate part
        # Model variables
    r = cp.zeros((nsteps+1,2), dtype=float32);
    r[0,] = r_0;

    # corr (dW_r_i, dZ_r_i) = rho_r_i
    dW_r_1 = cp.random.randn(nsteps,2, dtype=float32);
    dW_r_2 = cp.random.randn(nsteps,2, dtype=float32);
    dW_r_3 = cp.matmul(dW_r_1,cp.diag(rho_r)) + cp.matmul(dW_r_2,cp.diag((1-rho_r**2)**0.5));

    dW_r = dW_r_1 * cp.sqrt(dt);
    dZ_r = dW_r_3 * cp.sqrt(dt);

    r_ref = cp.zeros((1,2));
        # dr = lambda * (r_bar - r) * dt + eta * r^alpha * dZ_r
    for steps in range(nsteps):
        r[steps+1,] = cp.maximum(r[steps,] + cp.matmul((r_bar - r[steps,]) , cp.diag(Lambda)) * dt  + cp.matmul(cp.sqrt(r[steps,]) * dZ_r[steps,], cp.diag(eta)),r_ref);

    sum_v_1 = cp.zeros(nsteps, dtype=float32);
    sum_v_2 = cp.zeros(nsteps, dtype=float32);
    sum_r_1 = cp.zeros(nsteps, dtype=float32);
    sum_r_2 = cp.zeros(nsteps, dtype=float32);
    mu = cp.zeros(nsteps, dtype=float32);
    x = cp.zeros((nsteps+1,1), dtype=float32);

    # Valuation for dx
    for steps in range(nsteps):
        # Block for MuYu
        # sum_v_1(steps) = (a_i(1)^2 - a_j(1)^2) * v_1(steps) + (a_i(2)^2 - a_j(2)^2) * v_2(steps);
        sum_v_1[steps] = cp.matmul(v[steps,] , (a_i**2 - a_j**2));
        # sum_r_1(steps) = b_i^2 * r_i(steps)^(2*param_alpha) - b_j^2 * r_j(steps)^(2*param_alpha);
        sum_r_1[steps] = cp.matmul(r[steps,] , (b_i**2 - b_j**2));

        mu[steps] = r[steps,0] - r[steps,1] + 0.5 * (sum_v_1[steps] + sum_r_1[steps]);

            # Block for v
        sum_v_2[steps] = cp.matmul(v[steps,:]**0.5 * dW_v[steps,] , (a_i-a_j));###

            # Block for r
        sum_r_2[steps] = cp.matmul(r[steps,]**param_alpha * dW_r[steps,],(b_i-b_j));###

            # dx = (r_0 - r_i)*dt - a_i * diag_v^0.5 * dW_v - b_i * diag_r^alpha * dW_r
        x[steps+1] = x[steps] + mu[steps]*dt + sum_v_2[steps] + sum_r_2[steps];

        # Calculate the Smax and Smin

    S_end = S0*cp.exp(x[-1]);

    payoffs_call = max(S_end - K,0);
    payoffs_put = max(K - S_end,0);

    VcMCb = cp.exp(-r_0[1]*T)*payoffs_call;
    VpMCb = cp.exp(-r_0[1]*T)*payoffs_put;
    return cp.array([np.asscalar(VcMCb),np.asscalar(VpMCb)]);

In [17]:
# Monte Carlo
dt = T/nsteps;
output = cp.zeros((npaths,2),dtype=cp.float32)
for path in range (npaths):
    # cp.append(output,simulation_path(d,dt,v_0,r_0,rho_r,rho_v,nsteps,S0,K,eta,gamma))
    output[path,:] = simulation_path(d,dt,v_0,r_0,rho_r,rho_v,nsteps,S0,K,eta,gamma);
output.mean()

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [29]:
simulation_path(d,dt,v_0,r_0,rho_r,rho_v,nsteps,S0,K,eta,gamma)

C:\Users\Jinsong\AppData\Local\Temp\ipykernel_22464\248037717.py:70: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  return cp.array([np.asscalar(VcMCb),np.asscalar(VpMCb)]);


TypeError: no implementation found for 'numpy.asscalar' on types that implement __array_function__: [<class 'cupy._core.core.ndarray'>]